# IRIS EmbeddedPython

In [2]:
import iris

#Pandas dataframes
import pandas as pd
pd.options.display.max_rows = 9


In [3]:
print('\nInterSystems IRIS version:')
irisVersion = iris.cls('%SYSTEM.Version').GetVersion()
print(irisVersion)



InterSystems IRIS version:
IRIS for UNIX (Ubuntu Server LTS for x86-64 Containers) 2022.1 (Build 152U) Fri Mar 18 2022 01:14:43 EDT


# SQL
iris.sql.exec('select * from ...')

In [4]:
statement = iris.sql.exec('SELECT * FROM %SYS.ProcessQuery')
df = statement.dataframe()
jsn = df.to_json
print(jsn)
#df
df.dtypes
print(df.columns)
print(df.size)


<bound method NDFrame.to_json of       id namespace                routine  linesexecuted  globalreferences  \
0    393                          CONTROL              0                 0   
1    434                           WRTDMN            327              2041   
2    435                           GARCOL              0                 0   
3    436                           JRNDMN           2082               404   
..   ...       ...                    ...            ...               ...   
33   931      %SYS  %Net.Remote.Service.1          10955                82   
34  2512      %SYS      %SYS.WorkQueueMgr            441                36   
35  2513      %SYS      %SYS.WorkQueueMgr            518                51   
36  2535      %SYS      %SYS.WorkQueueMgr         530889             64787   

   state pidexternal username clientipaddress clientnodename  ...  \
0    RUN         393                                          ...   
1   RUNW         434                            

# SQL

In [5]:
statement = iris.sql.exec('SELECT * FROM Titanic_Table.Passenger')
df = statement.dataframe()
df

,id,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
...,...,...,...,...,...,...,...,...,...,...,...,...
887,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,0,1,2,W./C. 6607,23.4500,,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30.0000,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32,0,0,370376,7.7500,,Q


# IRIS persistent classes
% has to be translated to _ in python!

```
Class Sample.Company Extends (%Persistent, %Populate, %XML.Adaptor)
{

/// The company's name.
Property Name As %String(MAXLEN = 80, POPSPEC = "Company()") [ Required ];

/// The company's mission statement.
Property Mission As %String(MAXLEN = 200, POPSPEC = "Mission()");

/// The unique Tax ID number for the company.
Property TaxID As %String [ Required ];

/// The last reported revenue for the company.
Property Revenue As %Integer;

/// The Employee objects associated with this Company.
Relationship Employees As Employee [ Cardinality = many, Inverse = Company ];

}
```

In [6]:
myCompany = iris.cls('ObjectScript.Sample.Company')._New()
myCompany.Name = 'Acme Widgets Webinar!, Inc.'
myCompany.TaxID = '123456789'
status = myCompany._Save()
print (status)

1


In [7]:
print(myCompany._Id())


5


In [8]:
yourCompany = iris.cls("ObjectScript.Sample.Company")._OpenId(4)

In [9]:
print(yourCompany.Name)

Acme Widgets Webinar!, Inc.


## Add a method to our class:
```
Method Print() [ Language = python ]
{
    print ('\nName: ' + self.Name + ' TaxID: ' + self.TaxID)
}
```

In [10]:
yourCompany.Print()


Name: Acme Widgets Webinar!, Inc. TaxID: 123456789


# Work with Globals

In [33]:
myGref = iris.gref('^Workdays')

In [34]:
myGref[None] = 5

In [35]:
myGref[1] = 'Monday'

In [36]:
myGref[2] = 'Tuesday'

In [37]:
myGref[3] = 'Wednesday'

In [40]:
myGref[4] = 'Thursday'

In [41]:
myGref[5] = 'Friday'

In [13]:
print(myGref[3])

NameError: name 'myGref' is not defined

In [14]:
# Store procedures in Python

```SQL
CREATE FUNCTION tzconvert(dt DATETIME, tzfrom VARCHAR, tzto VARCHAR)
    RETURNS DATETIME
    LANGUAGE PYTHON
{
    from datetime import datetime
    from dateutil import parser, tz
    d = parser.parse(dt)
    if (tzfrom is not None):
        tzf = tz.gettz(tzfrom)
        d = d.replace(tzinfo = tzf)
    return d.astimezone(tz.gettz(tzto)).strftime("%Y-%m-%d %H:%M:%S")
}
```

In [21]:
convert = iris.sql.exec("SELECT tzconvert(now(), 'US/Eastern', 'UTC')")
print (convert)
df = convert.dataframe()
jsn = df.to_json
print(jsn)


<iris.%SYS.Python.SQLResultSet object at 0x7f90b1ae8390>
<bound method NDFrame.to_json of          expression_1
0 2022-03-28 17:28:36>
